In [1]:
from tensorflow.keras.models import load_model
from functions import margin_loss_fun_factory,DataGenerator,PredictGenerator,\
                        l2_distance_np,top_k,loop_distance,MAP,generate_feature,dot_distance_neg_np
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
import time
import tensorflow as tf
import numpy as np
from functools import partial

In [2]:
color = False
#FFA_size = 4

Load model

In [3]:
feature_model = load_model('Models/feature_model_DenseNet_greyscale_cross_entropy.h5')

In [4]:
#feature_model = load_model('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5', \
#                            custom_objects={'loss': margin_loss_fun_factory(0.1,5)})

In [4]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [5]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform = create_transform(aug)    

In [26]:
class TestGenerator(tf.keras.utils.Sequence):
    # use all images available and no TTA
    def __init__(self, Ids, batchSize):
        self.length = [len(sublist) for sublist in Ids]
        self.Ids = [item for sublist in Ids for item in sublist] # flatten list of lists, to [w1_img1,w1_img2...,w2_img1,w2_img2...]
        self.batchSize = batchSize
        
    def __len__(self):
        'Denotes the number of batches per epoch.'
        len_ = len(self.Ids)
        return len_//self.batchSize + (1 if len_%self.batchSize>0 else 0)

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.Ids[index*self.batchSize:(index+1)*self.batchSize]
        X = self.__data_generation(indexes)
        return X
        
    def __data_generation(self, indexes):
        return np.array([np.load(img) for img in indexes])[:,:,:,np.newaxis]
    
def generate_feature_test(Ids,batchSize,feature_model):
    feature_gen = TestGenerator(Ids if isinstance(Ids,list) else Ids.Imgs.tolist(),batchSize)
    feature = feature_model.predict_generator(feature_gen,workers=2,use_multiprocessing=True)
    return feature

In [20]:
feature_gen = TestGenerator(Ids_val.Imgs.tolist(),16)

In [23]:
feature_train = TestGenerator(Ids_train.Imgs.tolist(),16)

In [14]:
a = [1,2,3,4]

In [18]:
a[7:12] == []

True

In [12]:
len(feature_gen)//feature_gen.batchSize

127

In [13]:
127*feature_gen.batchSize

2032

In [11]:
sum(feature_gen.length)

2035

In [21]:
feature_val=feature_model.predict_generator(feature_gen,workers=2,use_multiprocessing=True)

In [27]:
feature_train=generate_feature_test(Ids_train,16,feature_model)

In [9]:
aggFuns = [partial(np.quantile,q=pct,axis=(1,2)) for pct in [0.1,0.25,0.5]] + [partial(np.mean,axis=(1,2))]    

In [10]:
FFA_sizes = [4,8]
results = []

In [11]:
for FFA_size in FFA_sizes:
    feature_train = generate_feature(Ids_train,transform,FFA_size,color,feature_model)
    feature_val = generate_feature(Ids_val,transform,FFA_size,color,feature_model)
    for aggFun in aggFuns:
        predicts = loop_distance(feature_train,feature_val,dot_distance_neg_np,aggFun)
        mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
        labels = Ids_val.Id.map(mapping_dict)
        score = MAP(labels,predicts)
        print([FFA_size,aggFun.keywords,score])
        results.append([FFA_size,aggFun.keywords,score])

[4, {'q': 0.1, 'axis': (1, 2)}, 0.04010513296227582]
[4, {'q': 0.25, 'axis': (1, 2)}, 0.033178726035868904]
[4, {'q': 0.5, 'axis': (1, 2)}, 0.02694805194805195]
[4, {'axis': (1, 2)}, 0.026004947433518864]
[8, {'q': 0.1, 'axis': (1, 2)}, 0.03877551020408165]
[8, {'q': 0.25, 'axis': (1, 2)}, 0.029359925788497217]
[8, {'q': 0.5, 'axis': (1, 2)}, 0.020748299319727888]
[8, {'axis': (1, 2)}, 0.02051638837353123]


In [ ]:
[4, {'q': 0.1, 'axis': (1, 2)}, 0.04010513296227582]
[4, {'q': 0.25, 'axis': (1, 2)}, 0.033178726035868904]
[4, {'q': 0.5, 'axis': (1, 2)}, 0.02694805194805195]
[4, {'axis': (1, 2)}, 0.026004947433518864]
[8, {'q': 0.1, 'axis': (1, 2)}, 0.03877551020408165]
[8, {'q': 0.25, 'axis': (1, 2)}, 0.029359925788497217]
[8, {'q': 0.5, 'axis': (1, 2)}, 0.020748299319727888]
[8, {'axis': (1, 2)}, 0.02051638837353123]